<a href="https://colab.research.google.com/github/seoho0529/TensorFlow/blob/main/tf27text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# RNN을 이용한 텍스트 생성
# 문맥을 반영해서 다음 단어를 예측하여 텍스트를 생성 - 다항분류 (many to one에 해당)
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical
import numpy as np
from keras.layers import Embedding, Dense, LSTM, Flatten
from keras.models import Sequential
'''
text = """거리에 하얀 눈이 쌓이고 있다
그의 눈이 밝게 빛난다
눈이 내리고 있는 오후 거리는 눈이 부시다"""  # label은 문장의 마지막에 있는 단어, ex) feature:거리, 하얀눈:label
'''
text = """
아파트 시장이 '숨고르기'에 들어간 가운데 부촌 1위인 서초는 물론 강남권에서도 수억 떨어진 거래가 잇따르고 있다. 일주일 새 매매가가 12억원 하락한 사례도 나왔다.
4일 국토교통부 실거래가 시스템과 아실에 따르면 최근 일주일 동안 매매가격이 최고가 대비 가장 많이 하락한 단지는 서울 강남구 도곡동의 '도곡렉슬'로 조사됐다. 이 아파트 전용 134.9㎡가 지난달 29일 37억2000만원에 거래됐다. 이는 2022년 5월 거래된 같은 면적 최고가(49억4000만원)보다 12억2000만원 내려간 것. 직전 거래인 9월(40억원)과 비교해도 2억8000만원 떨어졌다.
‘부촌 1번지’ 서초구도 예외는 아니다. 반포동 '래미안원베일리' 전용84㎡가 최근 38억5000만원에 거래됐다. 이는 지난 9월 거래가격인 43억원에 비해 4억5000만원 하락한 것이다. 지난 9월 29억3000만원에 주인이 바뀐 잠원동 '래미안신반포팰리스' 전용84㎡도 지난달 28억4000만원에 매매돼 두달 사이에 9000만원 떨어졌다.
송파구에서도 잠실동 '잠실엘스' 전용 84.88㎡가 지난달 27일 22억9000만원에 거래돼 2021년 9월 기록했던 최고가(27억원)보다 4억1000만원 하락했다. 이뿐만이 아니다. 부산 등 지방 랜드마크 아파트에서도 수억 하락한 거래가 포착되고 있다.
아실에 따르면 최근 일주일 동안 실거래 가격이 최고가 대비 3억원 이상 내린 단지는 전국에서 20여곳에 이른다. 2억원 이상 하락한 단지를 포함하면 60곳이 넘었다. 한국부동산원 통계를 보면 서울 강남 3구를 중심으로 대도시지역에서 하락폭이 커지고 있다.
박원갑 KB국민은행 부동산 수석전문위원은 “올해 강남권 회복세가 다른 지역에 비해서 가파르게 진행되면서 현재는 가격 메리트가 떨어진 점이 가장 결정적이라고 볼 수 있다”고 분석했다.
매매시장도 빠르게 냉각되고 있다. 직방에 따르면 올해 전국 아파트 매매거래 회전율은 3.04%로 2022년 2.28%에 비해 0.76%p 상승했다. 실거래 신고가 최초 도입된 2006년 이후 장기 시계열을 살펴보면 지난해(2.28%) 이후 역대 두번째로 낮은 수치를 기록했다. 아파트 매매 거래 회전율이 5%이하를 기록한 경우는 2022년과 2023년 뿐이다. 회전율은 아파트 재고 가구수 대비 실제 매매 거래된 해당 아파트 거래량 비율을 뜻한다. .
함영진 직방 빅데이터랩장은 “올 상반기 반짝 회복된 일부지역의 아파트 매매거래 회복흐름이 하반기 들어 다시 주춤한 모습이다”라며 "고금리에 경기둔화와 주택가격 부담에 대한 우려로 위축세가 뚜렷하다”고 말했다.
"""
tok = Tokenizer()
tok.fit_on_texts([text])
encoded = tok.texts_to_sequences([text])[0]
print(encoded)
print(tok.word_index)  # {'눈이': 1, '거리에': 2, '하얀': 3, '쌓이고': 4, '있다': 5, '그의': 6, '밝게': 7 ...

vocab_size = len(tok.word_index) + 1  # 0이 앞에 붙어있기 때문에 1을 더해줘야함

[1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46, 3, 6, 47, 48, 49, 2, 50, 51, 52, 53, 54, 55, 14, 7, 8, 6, 15, 56, 4, 9, 16, 57, 2, 17, 18, 58, 59, 60, 61, 62, 1, 19, 63, 64, 10, 65, 66, 20, 21, 22, 67, 23, 68, 69, 4, 70, 24, 71, 72, 73, 74, 75, 5, 76, 77, 78, 79, 25, 80, 81, 82, 83, 26, 84, 85, 86, 8, 87, 20, 21, 27, 5, 88, 89, 28, 90, 2, 91, 27, 5, 92, 93, 94, 95, 96, 97, 10, 98, 99, 100, 101, 102, 25, 103, 104, 105, 19, 106, 107, 10, 108, 109, 110, 111, 5, 112, 4, 113, 24, 114, 115, 116, 26, 117, 118, 119, 120, 121, 11, 2, 13, 122, 3, 14, 7, 8, 6, 15, 29, 123, 4, 9, 124, 30, 125, 17, 126, 127, 128, 129, 30, 2, 130, 131, 132, 133, 134, 135, 136, 18, 137, 138, 139, 140, 141, 142, 3, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 12, 158, 16, 159, 160, 161, 162, 163, 164, 165, 166, 3, 167, 7, 168, 169, 1, 31, 32, 170, 171, 172, 22, 33, 34, 173, 28, 174, 175, 176, 177, 29, 178, 179, 180, 181, 35, 182, 183, 184, 185, 33, 34, 35, 186, 187, 188, 189, 190,

In [2]:
# 훈련 데이터
sequences = list()
for line in text.split('\n'):  # 문장 토큰화, 엔터를 기준으로 잘랐기에 line은 3개
  enco = tok.texts_to_sequences([line])[0]
  # print(enco)  # [2, 3, 1, 4, 5] [6, 1, 7, 8] [1, 9, 10, 11, 12, 1, 13]
  for i in range(1, len(enco)):
    sequ = enco[:i + 1]
    # print(sequ)
    sequences.append(sequ)

print('학습에 참여할  샘플 수 : %d'%len(sequences))
print(sequences)
max_len = max(len(i) for i in sequences)  # 요소의 개수가 제일 긴게 7

psequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(psequences)

x = psequences[:, :-1]
y = psequences[:, -1]
print(x)
print(y)

y = to_categorical(y, num_classes=vocab_size)
print(y[:2])

학습에 참여할  샘플 수 : 276
[[1, 37], [1, 37, 38], [1, 37, 38, 39], [1, 37, 38, 39, 40], [1, 37, 38, 39, 40, 41], [1, 37, 38, 39, 40, 41, 42], [1, 37, 38, 39, 40, 41, 42, 43], [1, 37, 38, 39, 40, 41, 42, 43, 44], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46, 3], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46, 3, 6], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46, 3, 6, 47], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46, 3, 6, 47, 48], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46, 3, 6, 47, 48, 49], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46, 3, 6, 47, 48, 49, 2], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46, 3, 6, 47, 48, 49, 2, 50], [1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 11, 12, 13, 46

In [3]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_len - 1))
model.add(LSTM(32, activation='tanh'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(x, y, epochs=200, verbose=2)
print('model.evaluate : ', model.evaluate(x,y))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 64, 32)            7360      
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 230)               7590      
                                                                 
Total params: 25382 (99.15 KB)
Trainable params: 25382 (

In [4]:
# 문자열 생성
def sequence_gen_func(model, t, current_word, n):
  init_word = current_word
  sentence = ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]  # 매트릭스를 벡터로 바꿈
    encoded = pad_sequences([encoded], maxlen=max_len - 1, padding='pre')
    result = np.argmax(model.predict(encoded, verbose=0), axis=-1)

    # 예측 단어 찾기
    for word, index in t.word_index.items():
      # print(word, index)
      if index == result: # 예측한 단어의 인덱스와 동일한 단어가 있다면
        break # 해당 단어가 예측단어 이므로 break

    current_word = current_word + ' ' + word
    sentence = sentence + ' ' + word
  sentence = init_word + sentence
  return sentence

# print(sequence_gen_func(model, tok, '눈이', 5))  # '눈이' 뒤에 5개의 단어가 나옴
# print(sequence_gen_func(model, tok, '그의', 3))
# print(sequence_gen_func(model, tok, '그의', 10))
# print(sequence_gen_func(model, tok, '거리에', 3))

print(sequence_gen_func(model, tok, '매매', 5))  # '눈이' 뒤에 5개의 단어가 나옴
print(sequence_gen_func(model, tok, '아파트', 4))
print(sequence_gen_func(model, tok, '가격', 6))
print(sequence_gen_func(model, tok, '거래량', 7))

매매 국토교통부 실거래가 시스템과 아실에 따르면
아파트 시장이 '숨고르기'에 들어간 가운데
가격 1번지’ 서초구도 예외는 아니다 반포동 '래미안원베일리'
거래량 들어간 가운데 부촌 1위인 서초는 물론 강남권에서도
